In [39]:
import requests
import csv
import mysql.connector
import json
import uuid

In [40]:
debug=True

In [41]:
#get csv files from google
#Lab Data
response = requests.get('https://docs.google.com/spreadsheets/d/1pBmIyPgVe0B3i91tMO1_4Da0X5HF8ptlM5nNn2t9owY/export?format=csv&id=1pBmIyPgVe0B3i91tMO1_4Da0X5HF8ptlM5nNn2t9owY&gid=2035162726')
assert response.status_code == 200, 'Wrong status code'
lab_str=response.content.decode('utf8')
if debug:
    print(lab_str)
    print('-'*20)
response = requests.get('https://docs.google.com/spreadsheets/d/1JZvWCkz5PmIF11oSX_ENVgs1rjt46Vk2aRaCo-2_noI/export?format=csv&id=1JZvWCkz5PmIF11oSX_ENVgs1rjt46Vk2aRaCo-2_noI')
assert response.status_code == 200, 'Wrong status code'
tool_str=response.content.decode('utf8')
if debug:
    print(tool_str)


Timestamp,Lab Name,Building Name,Room Number,Manager(s),Description,Open Hours,Lat,Long
,Washburn Welding Shop,Washburn Shops and Stoddard Laboratories,Welding Shop,"Ian Anderson, James Loiselle, Toby Bergstrom",Metal Fabrication Shop,24/7,42.274089,-71.807191
,Washburn 105,Washburn Shops and Stoddard Laboratories,WB 105,"Ian Anderson, James Loiselle, Toby Bergstrom",Computer Lab and Metrology,24/7,42.274089,-71.807191
,Washburn 107,Washburn Shops and Stoddard Laboratories,WB 107,"Ian Anderson, James Loiselle, Toby Bergstrom",CNC Techning Center,24/7,42.274089,-71.807191
,Washburn 108,Washburn Shops and Stoddard Laboratories,WB 108,"Ian Anderson, James Loiselle, Toby Bergstrom",Prototyping Machine Shop,24/7,42.274089,-71.807191
,Higgins Machine Shop ,Higgins Laboratories,HL 004 & HL 005,"James Loiselle, Ian Anderson",Metal Machine Shop ,24/8,42.274342,-71.808187
,Foisie Innovation Studio Makerspace,Foisie Innovation Studio ,Floor 1,Adam Sears,Makerspace,7:30AM–1AM,42.274312,-71.808687


In [42]:
#open cvs
reader = csv.reader(lab_str.splitlines())
data=[d for d in reader]
names=data.pop(0)
lab_data=[{names[i]:v for i,v in enumerate(d)} for d in data]
if debug:
    [print(d) for d in lab_data]
    print('-'*20)

reader = csv.reader(tool_str.splitlines())
data=[d for d in reader]
names=data.pop(0)
tool_data=[{names[i]:v for i,v in enumerate(d)} for d in data]
if debug:
    [print(d) for d in tool_data]

{'Timestamp': '', 'Lab Name': 'Washburn Welding Shop', 'Building Name': 'Washburn Shops and Stoddard Laboratories', 'Room Number': 'Welding Shop', 'Manager(s)': 'Ian Anderson, James Loiselle, Toby Bergstrom', 'Description': 'Metal Fabrication Shop', 'Open Hours': '24/7', 'Lat': '42.274089', 'Long': '-71.807191'}
{'Timestamp': '', 'Lab Name': 'Washburn 105', 'Building Name': 'Washburn Shops and Stoddard Laboratories', 'Room Number': 'WB 105', 'Manager(s)': 'Ian Anderson, James Loiselle, Toby Bergstrom', 'Description': 'Computer Lab and Metrology', 'Open Hours': '24/7', 'Lat': '42.274089', 'Long': '-71.807191'}
{'Timestamp': '', 'Lab Name': 'Washburn 107', 'Building Name': 'Washburn Shops and Stoddard Laboratories', 'Room Number': 'WB 107', 'Manager(s)': 'Ian Anderson, James Loiselle, Toby Bergstrom', 'Description': 'CNC Techning Center', 'Open Hours': '24/7', 'Lat': '42.274089', 'Long': '-71.807191'}
{'Timestamp': '', 'Lab Name': 'Washburn 108', 'Building Name': 'Washburn Shops and Stod

In [43]:
#db_connections
mydb = mysql.connector.connect(
  host="makermap.cbeezzrvvyp6.us-east-2.rds.amazonaws.com",
  user="admin",
  passwd="makermap",
  database="innodb"
)

cursor = mydb.cursor()

In [44]:
# add building
def addBuilding(name, addr, lat, longi, desc, img, man, hour,uuid=uuid.uuid4()):
    sql_cmd = ("INSERT INTO Buildings(idBuildings, name, address, latitude, longitude, description, imageFolder, manager, hours) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)")
    data = (str(uuid), name, addr, lat, longi, desc, img, man, hour)
    cursor.execute(sql_cmd, data)
    print(cursor.statement)
    mydb.commit()

#add lab
def addLab(idB, name, hour, lat, longi, desc, room, img, man,uuid=uuid.uuid4()):
    sql_cmd = ("INSERT INTO Labs(idLabs, buildingID, name, hours, latitude, longitude, description, roomNum, imageFolder, manager) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
    data = (str(uuid), str(idB), name, hour, lat, longi, desc, room, img, man)
    cursor.execute(sql_cmd, data)
    print(cursor.statement)
    mydb.commit()
    
#add tool
def addTool(name, idL, desc, img, book, own, res):
    sql_cmd = ("INSERT INTO Tools(idTools, name, labID, description, imageFolder, bookingLink, ownerID, restrictPublic) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)")
    data = (str(uuid.uuid4()), name, str(idL), desc, img, book, own, res)
    cursor.execute(sql_cmd, data)
    print(cursor.statement)
    mydb.commit()

In [45]:
pushed_buildings={}
pushed_labs={}

In [46]:
for lab in lab_data:
    if lab['Lab Name'] != '':
        
        if lab['Building Name'] not in pushed_buildings.keys():
            b_uuid=str(uuid.uuid4())
            addBuilding(lab['Building Name'],'null',lab['Lat'],lab['Long'],'null','null','null',lab['Open Hours'],b_uuid)
            pushed_buildings[lab['Building Name']] = b_uuid
        if lab['Lab Name'] not in pushed_labs.keys():
            lab_uuid=str(uuid.uuid4())
            addLab(pushed_buildings[lab['Building Name']],lab['Lab Name'],lab['Open Hours'],lab['Lat'],lab['Long'],'null',lab['Room Number'],'null',lab['Manager(s)'],lab_uuid)
            pushed_labs[lab['Lab Name']]= lab_uuid
    

INSERT INTO Buildings(idBuildings, name, address, latitude, longitude, description, imageFolder, manager, hours) VALUES ('e8929aba-35e5-4849-8f92-16c80a254ade','Washburn Shops and Stoddard Laboratories','null','42.274089','-71.807191','null','null','null','24/7')
INSERT INTO Labs(idLabs, buildingID, name, hours, latitude, longitude, description, roomNum, imageFolder, manager) VALUES ('ae5b02b2-4695-46ea-be1d-551394b592f4', 'e8929aba-35e5-4849-8f92-16c80a254ade', 'Washburn Welding Shop', '24/7', '42.274089', '-71.807191', 'null', 'Welding Shop', 'null', 'Ian Anderson, James Loiselle, Toby Bergstrom')
INSERT INTO Labs(idLabs, buildingID, name, hours, latitude, longitude, description, roomNum, imageFolder, manager) VALUES ('9f10b5cd-20d8-4ea5-ba99-de557f634bdd', 'e8929aba-35e5-4849-8f92-16c80a254ade', 'Washburn 105', '24/7', '42.274089', '-71.807191', 'null', 'WB 105', 'null', 'Ian Anderson, James Loiselle, Toby Bergstrom')
INSERT INTO Labs(idLabs, buildingID, name, hours, latitude, longi

In [47]:
for tool in tool_data:
    if tool['Tool'] != '':
        addTool(tool['Tool'],pushed_labs[tool['Lab']],tool['Description'],tool['Photo'],'null','null','false')
        

INSERT INTO Tools(idTools, name, labID, description, imageFolder, bookingLink, ownerID, restrictPublic) VALUES ('dccc4a0c-0178-4ba3-ab41-49fc854846bf', 'Miller Syncrowave 250 Tig/Arc Welder', 'ae5b02b2-4695-46ea-be1d-551394b592f4', 'Tig Welder', '', 'null', 'null', 'false')
INSERT INTO Tools(idTools, name, labID, description, imageFolder, bookingLink, ownerID, restrictPublic) VALUES ('1520777b-9825-4845-9b47-75c2b2d3e9ec', 'Miller Millermatic Mig Welder', 'ae5b02b2-4695-46ea-be1d-551394b592f4', 'Mig Welder', 'https://data.makermap-wpi.com/images/MillerMillermatic ', 'null', 'null', 'false')
INSERT INTO Tools(idTools, name, labID, description, imageFolder, bookingLink, ownerID, restrictPublic) VALUES ('0b7a086a-bc81-49bd-b0ab-159c4bbd64b5', 'Miller Millermatic 300 Mig Welder', 'ae5b02b2-4695-46ea-be1d-551394b592f4', 'Mig Welder', 'https://data.makermap-wpi.com/images/MillerMillermatic300', 'null', 'null', 'false')
INSERT INTO Tools(idTools, name, labID, description, imageFolder, booking

INSERT INTO Tools(idTools, name, labID, description, imageFolder, bookingLink, ownerID, restrictPublic) VALUES ('fc5ce641-6566-4d5f-b2ee-90e32d7e19cd', 'DoAll Vertical Bandsaw 1', '821aa9a5-183e-4949-9f48-ce0cfbc2c756', 'Vertical Bandsaw', 'https://data.makermap-wpi.com/images/DoAllVerticalBandsaw', 'null', 'null', 'false')
INSERT INTO Tools(idTools, name, labID, description, imageFolder, bookingLink, ownerID, restrictPublic) VALUES ('ccd72d21-d4e7-4dd8-a538-0f86b30585c9', 'DoAll Vertical Bandsaw 2', '821aa9a5-183e-4949-9f48-ce0cfbc2c756', 'Vertical Bandsaw', 'https://data.makermap-wpi.com/images/DoAllVerticalBandsaw', 'null', 'null', 'false')
INSERT INTO Tools(idTools, name, labID, description, imageFolder, bookingLink, ownerID, restrictPublic) VALUES ('ad1ff904-4a54-423e-9d74-1a3197c28fce', 'Baldor Non-Ferrous Scotch-Brite bench wheel', '821aa9a5-183e-4949-9f48-ce0cfbc2c756', 'Scotch-Brite Wheel', '', 'null', 'null', 'false')
INSERT INTO Tools(idTools, name, labID, description, image

KeyError: 'Higgins Machine Shop'

In [ ]:
print(pushed_labs)